In [1]:
from mols2grid import display
from torch.nn.functional import linear

# Přidání cesty k adresáři, kde se nachází tvůj skript
# sys.path.append(os.path.abspath('/Users/krynekt/DataspellProjects/Bakalarka/QSPRpred/qsprpred/extra/gpu/models'))
import sys, os
sys.path.insert(0, os.path.abspath('/Users/krynekt/DataspellProjects/Bakalarka/QSPRpred/'))
# Nyní můžeš naimportovat svůj skript
print(sys.path)
from qsprpred.extra.gpu.models.neural_network import STFullyConnected

['/Users/krynekt/DataspellProjects/Bakalarka/QSPRpred', '/Users/krynekt/DataspellProjects/Bakalarka/QSPRpred', '/Users/krynekt/anaconda3/envs/ml/lib/python311.zip', '/Users/krynekt/anaconda3/envs/ml/lib/python3.11', '/Users/krynekt/anaconda3/envs/ml/lib/python3.11/lib-dynload', '', '/Users/krynekt/anaconda3/envs/ml/lib/python3.11/site-packages', '/Users/krynekt/anaconda3/envs/ml/lib/python3.11/site-packages/setuptools/_vendor']
test


In [3]:
from neural_network import STFullyConnected
from torch import optim

In [4]:
import pandas as pd

df = pd.read_csv('../../../../tutorials/tutorial_data/A2A_LIGANDS.tsv', sep='\t')

df.head()

,SMILES,pchembl_value_Mean,Year
0,Cc1nn(-c2cc(NC(=O)CCN(C)C)nc(-c3ccc(C)o3)n2)c(...,8.68,2008.0
1,Nc1c(C(=O)Nc2ccc([N+](=O)[O-])cc2)sc2c1cc1CCCC...,4.82,2010.0
2,O=C(Nc1nc2ncccc2n2c(=O)n(-c3ccccc3)nc12)c1ccccc1,5.65,2009.0
3,CNC(=O)C12CC1C(n1cnc3c1nc(C#CCCCCC(=O)OC)nc3NC...,5.45,2009.0
4,CCCn1c(=O)c2c(nc3cc(OC)ccn32)n(CCCNC(=O)c2ccc(...,5.20,2019.0


In [5]:
from qsprpred.data import QSPRDataset

import os

os.makedirs("tutorial_output/data", exist_ok=True)

dataset = QSPRDataset(
    df=df,
    store_dir="tutorial_output/data",
    name="QuickStartDataset",
    target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
    random_state=42
)

dataset.getDF()

qsprpred - WARNING - Existing data set found, but also found a data frame in store. Refusing to overwrite data. If you want to overwrite data in store, set overwrite=True.


,SMILES,pchembl_value_Mean,Year,QSPRID,pchembl_value_Mean_original,Split_IsTrain
QSPRID,,,,,,
QuickStartDataset_0000,Cc1cc(C)n(-c2cc(NC(=O)CCN(C)C)nc(-c3ccc(C)o3)n...,8.68,2008.0,QuickStartDataset_0000,8.68,True
QuickStartDataset_0001,Nc1c(C(=O)Nc2ccc([N+](=O)[O-])cc2)sc2nc3c(cc12...,4.82,2010.0,QuickStartDataset_0001,4.82,True
QuickStartDataset_0002,O=C(Nc1nc2ncccc2n2c(=O)n(-c3ccccc3)nc12)c1ccccc1,5.65,2009.0,QuickStartDataset_0002,5.65,True
QuickStartDataset_0003,CNC(=O)C12CC1C(n1cnc3c(NCc4cccc(Cl)c4)nc(C#CCC...,5.45,2009.0,QuickStartDataset_0003,5.45,True
QuickStartDataset_0004,CCCn1c(=O)c2c(nc3cc(OC)ccn32)n(CCCNC(=O)c2ccc(...,5.20,2019.0,QuickStartDataset_0004,5.20,True
...,...,...,...,...,...,...
QuickStartDataset_4077,CNc1ncc(C(=O)NCc2ccc(OC)cc2)c2nc(-c3ccco3)nn12,7.09,2018.0,QuickStartDataset_4077,7.09,False
QuickStartDataset_4078,Nc1nc(-c2ccco2)c2ncn(C(=O)NCCc3ccccc3)c2n1,8.22,2008.0,QuickStartDataset_4078,8.22,True
QuickStartDataset_4079,Nc1nc(CSc2nnc(N)s2)nc(Nc2ccc(F)cc2)n1,4.89,2010.0,QuickStartDataset_4079,4.89,False


In [6]:
display(dataset.X.head())

MolGridWidget()

KeyError: "['SMILES'] not in index"

In [7]:
from qsprpred.data.descriptors.fingerprints import MorganFP
from qsprpred.data import RandomSplit

# Specifiy random split for creating the train (80%) and test set (20%)
rand_split = RandomSplit(test_fraction=0.2, dataset=dataset)

# calculate compound features and split dataset into train and test
dataset.prepareDataset(
    split=rand_split,
    feature_calculators=[MorganFP(radius=3, nBits=2048)],
)

print(f"Number of samples in train set: {len(dataset.y)}")
print(f"Number of samples in test set: {len(dataset.y_ind)}")

dataset.save()

Number of samples in train set: 3265
Number of samples in test set: 817


In [11]:
dataset.makeClassification()

2048


In [25]:
import pandas as pd
# print(dataset.X["MorganFP_MorganFP_2042"])
print(dataset.getDescriptors())

                        MorganFP_MorganFP_0  MorganFP_MorganFP_1  \
QSPRID                                                             
QuickStartDataset_0000                False                False   
QuickStartDataset_0001                False                False   
QuickStartDataset_0002                False                False   
QuickStartDataset_0003                False                False   
QuickStartDataset_0004                False                False   
...                                     ...                  ...   
QuickStartDataset_4077                False                False   
QuickStartDataset_4078                False                False   
QuickStartDataset_4079                False                False   
QuickStartDataset_4080                False                False   
QuickStartDataset_4081                False                 True   

                        MorganFP_MorganFP_2  MorganFP_MorganFP_3  \
QSPRID                                         

In [26]:
from dnn import DNNModel

import torch
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import r2_score
from torch.nn import functional as f
from sklearn.metrics import mean_squared_error
from neural_network import STFullyConnected
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# p = {"n_epochs": [200],
#                             "lr": [1e-4],
#                             "neurons_h1": [2048],
#                             "neurons_hx": [1024],
#                             "weight_decay": [1e-4],
#                             "dropout_frac": [0.0],
#                             "act_fun": [f.selu]}
x = STFullyConnected(
        n_dim=dataset.X.shape[1],  # počet vstupních neuronů (počet deskriptorů)
        n_class=1,  # regresní úloha (1 výstup)
        gpus=[],
        device=device,
        batch_size=256,
        patience=20,
        tol=1e-4,
        is_reg=True,
        extra_layer=True,
    )

test = DNNModel("test")
print(test.getDevice())

cpu


In [27]:

test.fit(dataset.X, dataset.y)
# print(mean_squared_error(test.predict(dataset.X_ind), dataset.y_ind))

TypeError: object of type 'NoneType' has no len()

In [7]:
print(dataset.X.shape[1])
print(dataset.hasFeatures)

2048
True


In [10]:
display(dataset.X.head())

MolGridWidget()

KeyError: "['SMILES'] not in index"

In [9]:




param_grid = ParameterGrid({"n_epochs": [200],
                            "lr": [1e-4],
                            "neurons_h1": [2048],
                            "neurons_hx": [1024],
                            "weight_decay": [1e-4],
                            "dropout_frac": [0.0],
                            "act_fun": [f.selu]})  #selu
val_accuracies = []
val_accuracies_r = []
ind = 0

for params in param_grid:
    ind += 1
    print(ind, '/', len(param_grid))
    model_nn = STFullyConnected(
        n_dim=dataset.X.shape[1],  # počet vstupních neuronů (počet deskriptorů)
        n_class=1,  # regresní úloha (1 výstup)
        gpus=[],
        device=device,
        batch_size=256,
        patience=20,
        tol=1e-4,
        is_reg=True,
        extra_layer=True,
        **params
    )
    print(model_nn.get_params())
    model_nn.fit(dataset.X, dataset.y)
    print(params["act_fun"], mean_squared_error(model_nn.predict(dataset.X_ind), dataset.y_ind))
    val_accuracies.append(mean_squared_error(model_nn.predict(dataset.X_ind), dataset.y_ind))
    val_accuracies_r.append(r2_score(model_nn.predict(dataset.X_ind), dataset.y_ind))









1 / 1
{'act_fun': <function selu at 0x14590a0c0>, 'batch_size': 256, 'device': device(type='cpu'), 'dropout_frac': 0.1, 'extra_layer': True, 'gpus': [], 'is_reg': True, 'lr': 0.0001, 'n_class': 1, 'n_dim': 2048, 'n_epochs': 200, 'neurons_h1': 2048, 'neurons_hx': 1024, 'patience': 20, 'tol': 0.0001, 'weight_decay': 0.0001}
<function selu at 0x14590a0c0> 0.591590194764539


In [12]:
print(val_accuracies)
print(val_accuracies_r)

[0.591590194764539]
[0.44612061891762367]


In [21]:
import numpy as np

best_params = param_grid[(np.argmin(val_accuracies))]
display(best_params)
display(min(val_accuracies))
display(val_accuracies)


ValueError: If using all scalar values, you must pass an index


* RELU 1.045242449663438
* GELU 1.012
* ELU 1.0803313948254016
* SiLU 1.054
* SELU 1.089
* Dropout: 0.05


1 / 8
* <function relu at 0x15eb3a160> 1.1250744327450628
2 / 8
* <function leaky_relu at 0x15eb3a5c0> 1.0938768673113628
3 / 8
* <built-in function gelu> 1.1638879556715451
4 / 8
* <function elu at 0x15eb3a3e0> 0.9410014660390317
5 / 8
* <function silu at 0x15eb3ad40> 1.264585029437087
6 / 8
* <function tanh at 0x15eb3ab60> 1.5572187279142176
7 / 8
* <function sigmoid at 0x15eb3ac00> 9.547899011065818
8 / 8
* <function selu at 0x15eb3a480> 0.7631453763567609


Base model:
* MSE <- 0.31
* R2 <- 0.68


In [19]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

model_nn = STFullyConnected(n_dim=dataset.X.shape[1],  # počet vstupních neuronů (počet deskriptorů)
                            n_class=1,  # regresní úloha (1 výstup)
                            gpus=[],
                            device=device,
                            batch_size=256,
                            patience=20,
                            tol=1e-4,
                            is_reg=True,
                            extra_layer=True,
                            dropout_frac=0.25,
                            **best_params)

print("MSE:", mean_squared_error(model_nn.predict(dataset.X_ind), dataset.y_ind))
print("R2:", r2_score(model_nn.predict(dataset.X_ind), dataset.y_ind))


MSE: 0.7164694139567255
R2: 0.2779364571343922


In [20]:
model_nn_noreg = STFullyConnected(
    n_dim=dataset.X.shape[1],  # počet vstupních neuronů (počet deskriptorů)
    n_class=1,  # regresní úloha (1 výstup)
    gpus=[],
    device=device,
    n_epochs=200,
    lr=1e-4,
    batch_size=256,
    patience=20,
    tol=1e-4,
    is_reg=True,
    neurons_h1=1000,
    neurons_hx=200,
    extra_layer=True,
    dropout_frac=0.25,
    weight_decay=0.0
)


In [21]:
model_nn_noreg.fit(dataset.X, dataset.y)

(STFullyConnected(
   (dropout): Dropout(p=0.25, inplace=False)
   (fc0): Linear(in_features=2048, out_features=1000, bias=True)
   (fc1): Linear(in_features=1000, out_features=200, bias=True)
   (fc2): Linear(in_features=200, out_features=200, bias=True)
   (fc3): Linear(in_features=200, out_features=1, bias=True)
   (criterion): MSELoss()
 ),
 0)

In [22]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print("MSE:", mean_squared_error(model_nn_noreg.predict(dataset.X_ind), dataset.y_ind))
print("R2:", r2_score(model_nn_noreg.predict(dataset.X_ind), dataset.y_ind))

MSE: 0.7215939088954494
R2: 0.279348695180174
